# Air in a Pipe

The Code structure is similar to the previous one. 
What's new is the euqation system - and that we don't only take the upstream cells for the flux anymore. 

### Initialization

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt

# input
n_cells = 100
xmin = -1.
xmax = 1.

# divide pipe into cells
cellwidth = (xmax-xmin)/n_cells
cell_centers = np.linspace(xmin + cellwidth/2., xmax-cellwidth/2., n_cells)

# define indices of different quantities. 
# This makes reading the code much easier 
MASS = 0
MOMT = 1
ENER = 2
VELO = 1
PRES = 2
n_variables = 3

# Initialize solution array - now has three entries per cell! 
u = np.zeros((n_cells, n_variables))
u[:,MASS] = 1.
u[:,MOMT] = 1.
u[:,ENER] = 2. 
wave_region = np.abs(cell_centers)<0.2
u[wave_region,MASS] = 1.1 + 0.1*np.cos(5*np.pi*cell_centers[wave_region])
u[wave_region,MOMT] = 1.1 + 0.1*np.cos(5*np.pi*cell_centers[wave_region])

# plot initial solution
plt.figure()
plt.bar(cell_centers, u[:,MASS], width=cellwidth, edgecolor='black')

In [ ]:
# define left and right cell neighbours (again)
cell_numbers = np.arange(n_cells)
left_neighbours = cell_numbers - 1
right_neighbours = (cell_numbers + 1)
right_neighbours[n_cells - 1] = 0 

### Helper functions

Here come some helper functions we haven't iscussed in the slides. They... 
- Compute velocity and pressure from the conserved variables
- Compute the largest possible time step
- compute the ominous flux stabilization term (draw the rest of the owl!)

In [ ]:
# calculate flux 
def velocity(u):
    vel = u[:,MOMT] / u[:,MASS]
    return vel

def pressure(u):
    pres = 0.4*(u[:,ENER] - 0.5* u[:,MASS] * velocity(u)**2)
    return pres

def sos(u): 
    return np.sqrt(1.4*pressure(u)/u[:,MASS])

def lmbda(u):
    return sos(u)+np.abs(velocity(u))

def calc_time_step(u): 
    return 0.99*cellwidth/np.max(lmbda(u))

def stabilization(u_left, u_right):
    a = np.maximum(lmbda(u_left), lmbda(u_right))
    return - 0.5 * a[...,np.newaxis] * (u_right-u_left)


### Core functions 

Your turn!
Tasks: 
- First fill in the flux function which calculates the flux from a single state $u$
- Fill in the flux over a side by calling the flux function twice and averaging it.

The time step routine is already filled in.


In [ ]:
# compute the flux function from one single state u
def flux_cell(u):
    f_cell = np.zeros_like(u)
    f_cell[:,MASS] = u[:,MASS] * velocity(u)
    f_cell[:,MOMT] = u[:,MOMT] * velocity(u) + pressure(u)
    f_cell[:,ENER] = u[:,ENER] * velocity(u) + (pressure(u) * velocity(u))
    return f_cell

# flux over a side between two cells
def flux_side(u_left, u_right): 
    f_left = flux_cell(u_left) 
    f_right = flux_cell(u_right) 
    f_average = 0.5 * (f_left + f_right) 
    f_side = f_average + stabilization(u_left, u_right)
    return f_side

# advance state u by one time step
def perform_time_step(u):
    time_step = calc_time_step(u)
    inflow  = flux_side(u[left_neighbours,:], u) * time_step
    outflow = flux_side(u, u[right_neighbours,:]) * time_step
    u_total = u * cellwidth
    u_total_new = u_total + (inflow - outflow)
    u = u_total_new / cellwidth
    return u


Let's let it run! (this just calls the time step routine in a loop and does all the plotting - nothing to do here!)

In [ ]:

# plotting stuff (again...) 
SHOW = MASS
fig, ax = plt.subplots(1, 4)
fig.set_size_inches(12, 3)
ax[0].bar(cell_centers, u[:,SHOW], width=cellwidth)

# time step loop with some plotting
for ifig in range(1,4):
    for _ in range(20):
        # this calls the time step!
        u = perform_time_step(u)
    ax[ifig].bar(cell_centers, u[:,SHOW], width=cellwidth)